In [2]:
# 同openrank处理情况相同

import pymysql
import json

# 连接到 MySQL 数据库
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='cptbtptp789',
    database='project_data',
    charset='utf8mb4'
)

cursor = conn.cursor()

# 创建新表 activity_detailed_metrics
create_table_query = """
CREATE TABLE IF NOT EXISTS activity_detailed_metrics (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    month VARCHAR(7) NOT NULL,
    value DECIMAL(10, 2)
)
"""
try:
    cursor.execute(create_table_query)
    print("表 activity_detailed_metrics 创建成功")
except pymysql.Error as e:
    print(f"创建表 activity_detailed_metrics 时出错: {e}")

# 从 activity_metrics 表中查询数据
select_query = "SELECT project_name, metric_value FROM activity_metrics"
cursor.execute(select_query)

# 获取查询结果
results = cursor.fetchall()

# 处理并插入数据到新表
for row in results:
    project_name, metric_value_str = row
    metric_value = json.loads(metric_value_str)
    for month_key, value in metric_value.items():
        if isinstance(value, (int, float)):
            # 处理月份键，只保留年月部分
            month = month_key[:7]
            insert_query = "INSERT INTO activity_detailed_metrics (project_name, month, value) VALUES (%s, %s, %s)"
            try:
                cursor.execute(insert_query, (project_name, month, value))
                conn.commit()
            except pymysql.Error as e:
                print(f"插入数据时出错: {e}")

# 关闭连接
cursor.close()
conn.close()

表 activity_detailed_metrics 创建成功
